<a href="https://colab.research.google.com/github/jogden33/HousingProject/blob/master/Housing_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Pandas and do some data exploration on the three datasets

In [86]:
import pandas as pd
import re as re
import numpy as np
import random

Housing = pd.read_csv("/housing-info.csv")
Income = pd.read_csv("/income-info.csv")
County = pd.read_csv("/zip-city-county-state.csv")

In [ ]:
Housing.replace(to_replace = r'^[A-Z]{4}$', value=np.nan, regex=True, inplace=True)

# Delete the rows where guid is NaN
Housing.dropna(axis = 0, subset = ['guid'], inplace = True)
Housing.head(25)

In [ ]:
Housing.loc[Housing['housing_median_age'].isnull(), 'housing_median_age'] = Housing['housing_median_age'].apply(lambda v: np.random.randint(10,50))
Housing.loc[Housing['total_rooms'].isnull(), 'total_rooms'] = Housing['total_rooms'].apply(lambda v: np.random.randint(1000, 2000))
Housing.loc[Housing['total_bedrooms'].isnull(), 'total_bedrooms'] = Housing['total_bedrooms'].apply(lambda v: np.random.randint(1000,2000))
Housing.loc[Housing['population'].isnull(), 'population'] = Housing['population'].apply(lambda v: np.random.randint(5000, 10000))
Housing.loc[Housing['households'].isnull(), 'households'] = Housing['households'].apply(lambda v: np.random.randint(500, 2500))
Housing.loc[Housing['median_house_value'].isnull(), 'median_house_value'] = Housing['median_house_value'].apply(lambda v: np.random.randint(100000, 250000))
Housing.head(25)


In [ ]:
CountyImputed = County[['zip_code', 'city', 'state', 'county']].copy()
CountyImputed.replace(to_replace = r'^[A-Z]{4}$', value=np.nan, regex=True, inplace=True)
CountyImputed.dropna(axis = 0, subset = ['zip_code'], inplace = True)
CountyImputed.sort_values(by=['state','city','county'], inplace = True)
CountyImputed['NewZip']=CountyImputed.zip_code.str[:1]+'0000'
CountyImputed.drop_duplicates(['state','city','county'], keep='last', inplace = True)
CountyImputed.tail(25)




In [ ]:
Income.head(25)

In [ ]:
Income.replace(to_replace = r'^[A-Z]{4}$', value=np.nan, regex=True, inplace=True)
Income.dropna(axis = 0, subset = ['guid'], inplace = True)
Income.loc[Income['median_income'].isnull(), 'median_income'] = Income['median_income'].apply(lambda v: np.random.randint(100000, 750000))
Income.head(25)

In [82]:
#County.replace(to_replace = r'^[A-Z]{4}$', value=np.nan, regex=True, inplace=True)
#County.dropna(axis = 0, subset = ['guid'], inplace = True)
#County.head(25)

In [ ]:
Housing_Zip = Housing.merge(County, how = 'outer', on = ['guid', 'zip_code'])
Housing_Zip.head(25)

In [ ]:
Housing_Zip_Income = Housing_Zip.merge(Income, how = 'outer', on = ['guid', 'zip_code'])
Housing_Zip_Income.head(25)

In [ ]:
# Step 4: Create a reference dataset with non-duplicate zip codes so we can impute missing Zip Codes
ZipTable = County[['zip_code', 'city', 'state', 'county']].copy()

# Next, replace all of the corrupted zip codes with NaN (missing value) and drop all of those rows
ZipTable.replace(to_replace = r'^[A-Z]{4}$', value=np.nan, regex=True, inplace=True)
ZipTable.dropna(axis = 0, subset = ['zip_code'], inplace = True)

# Finally, sort the data by state, city, county, in that order and create a new variable that strings the first
# number from the zip code to '0000'. Also, get rid of any duplicates.
ZipTable.sort_values(by=['state','city','county'], inplace = True)
ZipTable['NewZip']=ZipTable.zip_code.str[:1]+'0000'
ZipTable.drop_duplicates(['state','city','county'], keep='last', inplace = True)

# Now we have a clean reference dataset from which we can join our other two datasets to obtain zip codes for missing values
Housing_Zip_Income2 = Housing_Zip_Income.merge(ZipTable, how = 'left', on = ['state', 'city', 'county'])
Housing_Zip_Income2.tail(50)

In [134]:
Housing_Zip_Income2.loc[Housing_Zip_Income2['zip_code_x'].isnull(), 'zip_code_x'] = Housing_Zip_Income2["NewZip"]
Housing_Zip_Income2['id'] = range(1,1+len(Housing_Zip_Income2))
Housing_Zip_Income2.rename(columns = {'zip_code_x':'zip_code', 'housing_median_age':'median_age'}, inplace = True)
HousingData = Housing_Zip_Income2[['id','guid','zip_code', 'city','state','county','median_age','total_rooms','total_bedrooms','population','households','median_income','median_house_value']].copy()
HousingData.head(50)

,id,guid,zip_code,city,state,county,median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,1,008B672B-5BA5-6A55-498C-BA2474F3A5A1,76099,Grapevine,TX,Tarrant,38,3229,636,1599,609,386460,257100
1,2,00D8ABF2-5101-28E6-68F0-D2A572869349,70000,Dallas,TX,Dallas,21,2281,557,1510,460,286250,189600
2,3,04A897A1-46B4-8375-25CB-0B850C216814,88321,Encino,NM,Torrance,26,2249,344,1311,373,502870,247199
3,4,0B310FFB-1D63-79B9-2C47-8D92AE6F0F28,92230,Cabazon,CA,Riverside,26,1991,330,1014,304,430680,240100
4,5,0B702CD9-3C77-7E04-363C-6F2A5EEA072B,77025,Houston,TX,Harris,37,1756,345,836,335,437500,218200
5,6,0FB1FA49-79C0-0125-8223-75FAFA554C0C,85258,Scottsdale,AZ,Maricopa,15,10377,2331,4507,1807,224660,66800
6,7,10D2266D-8D12-2279-6D43-8F7A95FB1854,77474,Sealy,TX,Austin,44,2101,496,1038,500,610903,217900
7,8,117BAB66-5876-9089-4F56-755BD963992F,84106,Salt Lake City,UT,Salt Lake,19,4819,1492,2572,1613,233930,118200
8,9,11D47F49-9367-96A1-0B95-EF7D82900887,93247,Lindsay,CA,Tulare,35,2240,423,8900,396,317990,86700
9,10,12D539B1-876D-65F2-304F-574DAEEA76BB,77201,Houston,TX,Harris,31,2210,519,1047,472,242032,271300
